# generate json-ld for ag given graphml definition

In [1]:
import networkx as nx

# Load the graph from the provided graphml file
# fix the path to the graphml file, file path shoould use // instead of //


graph_path = 'C://Users//donbr//github//nsclc-pathways//ts_sl_gene_graph.graphml'
G = nx.read_graphml(graph_path)

# Get an overview of nodes and their attributes
nodes_data = G.nodes(data=True)

# Displaying the first 5 nodes to understand the structure and naming conventions
nodes_sample = list(nodes_data)[:5]

nodes_sample


[('TYMS',
  {'gene_type': 'SL_GENE',
   'hgnc_id': 'HGNC:12441',
   'name': 'thymidylate synthetase',
   'locus_group': 'protein-coding gene',
   'locus_type': 'gene with protein product',
   'status': 'Approved',
   'location': '18p11.32',
   'alias_symbol': 'Tsase,TMS,HsT422',
   'prev_symbol': 'TS',
   'date_approved_reserved': '1990-09-10',
   'date_modified': '2023-01-20',
   'entrez_id': '7298',
   'ensembl_gene_id': 'ENSG00000176890',
   'vega_id': 'OTTHUMG00000131473',
   'ucsc_id': 'uc010dka.2',
   'ena': 'X02308',
   'refseq_accession': 'NM_001071',
   'uniprot_ids': 'P04818',
   'mgd_id': 'MGI:98878',
   'rgd_id': 'RGD:3921',
   'lsdb': 'LRG_783|http://ftp.ebi.ac.uk/pub/databases/lrgex/LRG_783.xml',
   'omim_id': '188350',
   'orphanet': '241969.0',
   'iuphar': 'objectId:2642',
   'enzyme_id': '2.1.1.45',
   'agr': 'HGNC:12441',
   'mane_select': 'ENST00000323274.15,NM_001071.4',
   'uuid': 'c667dcf6-a2c1-48a7-ae57-aa4cc4023a2e'}),
 ('LKB1', {'gene_type': 'TS_GENE'}),
 ('NF

In [2]:
import networkx as nx
import json

# Load the GraphML file
# graphml_file_path = '/mnt/data/ts_sl_gene_graph.graphml'
# G = nx.read_graphml(graphml_file_path)
# pubmed may also be useful - 	https://identifiers.org/pubmed:16333295

# Define the JSON-LD context for linked data
context = {
    "name": "http://schema.org/name",
    "symbol": "http://identifiers.org/symbol/",
    "entrezId": "http://identifiers.org/ncbigene/",
    "hgncId": "http://identifiers.org/hgnc/",
    "ensemblGeneId": "http://identifiers.org/ensembl/",
    "uniprotId": "http://identifiers.org/uniprot/",
    "iupharId": "http://identifiers.org/iuphar/",
    "keggId": "http://identifiers.org/kegg.genes/",
    "omimId": "http://identifiers.org/omim/",
    "orphanetId": "http://identifiers.org/orphanet/",
    "location": "http://schema.org/position",
    "relationship": "http://schema.org/relatedTo"
}

# Prepare the JSON-LD graph structure
json_ld_graph = []

# Iterate through each node in the GraphML file (each gene)
for node_id, node_data in G.nodes(data=True):
    # Use the node_id directly as the gene symbol (e.g., "TYMS")
    gene_symbol = node_id

    # Start with an empty dictionary for gene_entry
    gene_entry = {}

    # Conditionally add attributes if they exist
    if 'hgnc_id' in node_data:
        gene_entry["@id"] = f"http://identifiers.org/hgnc/{node_data.get('hgnc_id', '').split(':')[-1]}"
    
    # Always add the symbol (based on node ID)
    gene_entry["symbol"] = gene_symbol  # Use the node ID directly as the symbol
    
    if 'name' in node_data:
        gene_entry["name"] = node_data.get('name', '')
    
    if 'entrez_id' in node_data:
        gene_entry["entrezId"] = f"http://identifiers.org/ncbigene/{node_data.get('entrez_id', '')}"
    
    if 'ensembl_gene_id' in node_data:
        gene_entry["ensemblGeneId"] = f"http://identifiers.org/ensembl/{node_data.get('ensembl_gene_id', '')}"
    
    if 'uniprot_ids' in node_data:
        gene_entry["uniprotId"] = f"http://identifiers.org/uniprot/{node_data.get('uniprot_ids', '').split(',')[0]}"
    
    if 'iuphar' in node_data:
        gene_entry["iupharId"] = f"https://www.guidetopharmacology.org/GRAC/ObjectDisplayForward?objectId={node_data.get('iuphar', '').split(':')[-1]}"
    
    if 'entrez_id' in node_data:
        gene_entry["keggId"] = f"http://identifiers.org/kegg.genes/hsa:{node_data.get('entrez_id', '')}"
    
    if 'omim_id' in node_data:
        gene_entry["omimId"] = f"https://omim.org/entry/{node_data.get('omim_id', '')}"
    
    if 'orphanet' in node_data:
        gene_entry["orphanetId"] = f"http://www.orpha.net/ORDO/Orphanet_{node_data.get('orphanet', '').split('.')[0]}"
    
    if 'location' in node_data:
        gene_entry["location"] = node_data.get('location', '')

    # Only add the gene entry if it contains data
    if gene_entry:
        json_ld_graph.append(gene_entry)

# Create the final JSON-LD structure
json_ld_data = {
    "@context": context,
    "@graph": json_ld_graph
}

# Convert the JSON-LD structure to a string
json_ld_output = json.dumps(json_ld_data, indent=2)

# Output the JSON-LD to a file (or print it out)
output_file_path = 'gene_network.jsonld'
with open(output_file_path, 'w') as f:
    f.write(json_ld_output)

print(f"JSON-LD output with HGNC-approved symbols saved to {output_file_path}")


JSON-LD output with HGNC-approved symbols saved to gene_network.jsonld
